# Génération des cliques à partir d'un échantillon et des classes

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [204]:
# -*- coding: utf8 -*-
import codecs,glob,re,pickle,features
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import networkx as nx
debug=False
from __future__ import print_function

In [205]:
import yaml

from ipywidgets import FloatProgress
from IPython.display import display

In [206]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [207]:
saut="\n"

### Préparation des matrices de traits

In [208]:
#features.add_config('../bdlexique.ini')
features.add_config('../german.ini')
fs=features.FeatureSystem('phonemes')

In [209]:
genDigraphe=False
genGraphe=False

In [210]:
phonologicalMap="-X"

repSample="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
repClasses="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
repSample="/Users/gilles/Box Sync/2015-Data/German/"
repClasses="/Users/gilles/Box Sync/2015-Data/German/"
serie="MSP"
numEchantillon=0
nomGold="/Users/gilles/Box Sync/2015-Data/MGC-170716-Adjectifs.pkl"
nomGold=""

listeSamples=glob.glob(repSample+"Longitudinal*%s.pkl"%serie)
listeSamples=glob.glob(repSample+"German*%s.pkl"%serie)
nomSample=listeSamples[numEchantillon]
print (nomSample)
listeClasses=glob.glob(repSample+"Longitudinal*%s-Regles.pkl"%serie)
listeClasses=glob.glob(repSample+"German*%s-Regles.pkl"%serie)
nomClasse=listeClasses[numEchantillon]
print (nomClasse)
analyseCases=[]

/Users/gilles/Box Sync/2015-Data/German/German-V-X-MSP.pkl
/Users/gilles/Box Sync/2015-Data/German/German-V-X-MSP-Regles.pkl


In [211]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}
def checkFrench(prononciation):
    result=recoder(prononciation)
    m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
    if m:
        print ("pb avec un glide final", prononciation)
    m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
    if m:
        n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
        if not n:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
    m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
    if m:
        glide=m.group(2)
        result=m.group(1)+dierese[glide]+m.group(3)
    return result

In [212]:
def lireLexique(nomLexique):
    with open(nomLexique, 'rb') as input:
        lexique=pickle.load(input)
    return lexique

In [213]:
def lexique2Paradigmes(lexique):
    return pd.pivot_table(lexique, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [214]:
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [215]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

### Préparation du paradigme de référence (Gold)

In [216]:
if nomGold:
    lexiqueGold=lireLexique(nomGold)
    lexiqueGold["phono"]=lexiqueGold["phono"].apply(checkFrench)
    paradigmesGold=lexique2Paradigmes(lexiqueGold)

### Préparation du paradigme de référence (Base)

In [217]:
def preparerLexiqueBase(nomSample):
    nomBase=nomSample
    if serie=="OMP":
        nomBase=nomSample.replace("OMP","MSP")
    print (nomBase)
    lexiqueBase=lireLexique(nomBase)
    lexiqueBase["phono"]=lexiqueBase["phono"].apply(checkFrench)
    paradigmesBase=lexique2Paradigmes(lexiqueBase)
    return paradigmesBase

### Préparation des cases du paradigme

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [218]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [219]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [220]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [221]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

# Classe pour la gestion des patrons, des classes et des transformations

In [222]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme, file=logfile) 
                print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [223]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [224]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [225]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [226]:
def preparerSample(nomSample):
    lexique=lireLexique(nomSample)
    lexique["phono"]=lexique["phono"].apply(checkFrench)
    morphomes={}
    if "morphome" in lexique.columns:
        for morphome in lexique.morphome.unique():
            if "/" in morphome:
                morphomeCases=morphome.split("/")
                morphomeCase=lexique[lexique["morphome"]==morphome]["case"].unique().tolist()
                if len(morphomeCase)>1:
                    print("pb",morphomeCase)
                else:
                    morphomeCase=morphomeCase[0]
                morphomes[morphomeCase]=morphomeCases
        print (morphomes)
    paradigmes=lexique2Paradigmes(lexique)
    paradigmes=paradigmes.dropna(axis=1,how='all')
    return morphomes,paradigmes

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [227]:
def preparerClasses(numSample):
    nomClasse=listeClasses[numSample]
    print(nomClasse)
    with open(nomClasse, 'rb') as input:
        resultatsLecture = pickle.load(input)
    return resultatsLecture
#resultatsLecture=preparerClasses(numSample)

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [228]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self):
        caseListe=analyseCases
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [229]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [230]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [231]:
def generateParadigms(generation1,genDigraphe=True):
#    print (generation1.formes)
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()
    for caseDepart in analyseCases:
#        print (distributionInitiale)
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

In [232]:
def generate(lexeme,genDigraphe=True):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

#### Calculer le score de la clique

In [233]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

## Génération des formes

In [234]:
def genererFormes(paradigmes):
    debug=False
    listeTest=paradigmes.dropna(thresh=1)["lexeme"].values.tolist()
    nbLexemes=len(listeTest)
    print (nbLexemes)
    globDigraphe=nx.DiGraph()
    globGraphe=nx.Graph()
    cliques=[]
    cliquesScores={}
    cliquesListes={}
    numClique=0
    #progressBar = FloatProgress(min=0, max=nbLexemes)
    #display(progressBar)
    for i,element in enumerate(listeTest):
    #    if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
    #    progressBar.value=i
        result=generate(element,genDigraphe)
        (generation,lexDigraphe,lexGraphe,lexCliques)= result
    #    print (generation,lexDigraphe,lexGraphe,lexCliques)
        if genDigraphe:
            globDigraphe=nx.union(globDigraphe,lexDigraphe)
        if genGraphe:
            globGraphe=nx.union(globGraphe,lexGraphe)
        cliques.extend(lexCliques)
        for clique in lexCliques:
            cliquesScores[numClique]=cliqueScore(clique,lexGraphe)
            cliquesListes[numClique]=clique
            numClique+=1
    versionStamp=dateheure()
    if genDigraphe: 
        nx.write_dot(globDigraphe,u"2015-Data/digraphe-%s.dot"%versionStamp)
    if genGraphe:
        nx.write_dot(globGraphe,u"2015-Data/graphe-%s.dot"%versionStamp)    
    return listeTest,cliques,cliquesScores

### Faire la liste des cases lexicalisées de l'échantillon

In [235]:
def preparerCasesLexicales(paradigmes):
    nbFormesLexicales={}
    casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
    for element in casesLexicales:
        casesLexicales[element].remove("lexeme")
        nbFormesLexicales[element]=len(casesLexicales[element])
    return casesLexicales
#casesLexicales=preparerCasesLexicales(paradigmes)

## Comparer la sortie des cliques avec le paradigme de départ

In [236]:
def filtrerFidelite(clique,casesLexicales):
    seuilClique=1
    paradigmesListe=[]
    #progressBar = FloatProgress(min=0, max=len(cliques))
    #display(progressBar)
    for n,clique in enumerate(cliques):
    #    progressBar.value=n
        if len(clique)>seuilClique:
            paradigmeClique={}
            sampleOK=True
            point=clique[0].split("-")
            lPoint=len(point)
            if lPoint==3:
                lexeme=point[0]
            else:
                lexeme="-".join(point[0:len(point)-2])
            paradigmeClique["lexeme"]=lexeme
            casesLexeme=casesLexicales[lexeme]
            nbInitial=len(casesLexeme)
            if casesLexeme and len(casesLexeme)<=len(clique):
                for element in casesLexeme:
                    champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
                    if ","  in champForme:
                        formes=champForme.split(",")
                        nbInitial+=len(formes)-1
                        okFormes=False
                        for forme in formes:
                            pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                            if pointCase in clique:
                                okFormes=True
                                if debug: print ("point",pointCase)
                        if okFormes:
                            sampleOK=True
                        else:
                            sampleOK=False
                            break
                    else:
                        forme=champForme
                        pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                        if debug: print (pointCase, clique)
                        if not pointCase in clique:
                            sampleOK=False
                            break
            else:
                sampleOK=False
            if sampleOK:
                for element in clique:
                    elements=element.split("-")
                    forme=elements[-2]
                    taminfo=elements[-1]
                    paradigmeClique[taminfo]=forme
                paradigmeClique["score"]=cliquesScores[n]
                paradigmeClique["ajouts"]=len(clique)-nbInitial
                paradigmeClique["lexicales"]=", ".join(casesLexeme)
                paradigmesListe.append(paradigmeClique)
            else:
                if debug:
                    print ()
                    print (lexeme,clique)
                    print ()
    paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=[u"lexeme"]+sampleCases+[u"score",u"ajouts","lexicales"])
    return paradigmesCLIQUES
#paradigmesCLIQUES=filtrerFidelite(clique,casesLexicales)

In [237]:
def mesurerSimilarite(x):
    def calculSimilariteCase(x,caseMorphome):
        if (paradigmesBase[paradigmesBase["lexeme"]==lexeme][caseMorphome]!=x[case]).all():
            if (paradigmesGold[paradigmesGold["lexeme"]==lexeme][caseMorphome]!=x[case]).all():
                if x[case]:
                    result["wrong"]+=1
                else:
                    result["missing"]+=1
            else:
                result["right"]+=1
        else:
            result["known"]+=1
    
    result={"known":0,"right":0,"wrong":0,"missing":0}
    lexeme=x["lexeme"]
#    print (lexeme,sampleCases)
    for case in sampleCases:
#        print (case)
        if case in morphomes:
#            print ("morphome",case,morphomes[case])
            for caseMorphome in morphomes[case]:
#                print ("caseMorphome",caseMorphome)
                calculSimilariteCase(x,caseMorphome)
#                print (result,case)
        else:
#            print ("normal",case)
            calculSimilariteCase(x,case)
#    print (lexeme,result["known"],result["right"],result["wrong"],result["missing"])
    return (result["known"],result["right"],result["wrong"],result["missing"])

In [238]:
def filtrerMax(paradigmesCLIQUES):
    paradigmesMAX=paradigmesCLIQUES[paradigmesCLIQUES["score"]==paradigmesCLIQUES.groupby(["lexeme"])["score"].transform(max)].reset_index()
    del paradigmesMAX["index"]
    paradigmesMAX["score"]=paradigmesMAX["score"].apply(str)
    paradigmesMAX["ajouts"]=paradigmesMAX["ajouts"].apply(str)

    paradigmesSILVER=paradigmesMAX.groupby("lexeme").agg(lambda x: ",".join(list(set(x.dropna().values)))).reset_index()
    if nomGold:
        paradigmesSILVER["eval"]=paradigmesSILVER.apply(lambda x: mesurerSimilarite(x), axis=1)
        paradigmesSILVER["known"],paradigmesSILVER["right"],paradigmesSILVER["wrong"],paradigmesSILVER["missing"]=zip(*paradigmesSILVER["eval"])
        paradigmesSILVER.drop("eval",axis=1,inplace=True)

    return paradigmesSILVER

#paradigmesSILVER=filtrerMax(paradigmesCLIQUES)


# Boucle sur les échantillons

In [239]:
for numSample,nomSample in enumerate(listeSamples):
    print (numSample,nomSample)
    paradigmesBase=preparerLexiqueBase(nomSample)
    morphomes,paradigmes=preparerSample(nomSample)
    sampleCases=paradigmes.columns.values.tolist()
    sampleCases.remove(u"lexeme")
    casesTotales=paradigmesGold.columns.values.tolist()
    casesTotales.remove(u"lexeme")
    resultatsLecture=preparerClasses(numSample)
    analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
    if sorted(analyseCases)!=sorted(casesTotales):
        print ("Attention l'analyse ne comprend pas toutes les cases")
    listeTest,cliques,cliquesScores=genererFormes(paradigmes)
    casesLexicales=preparerCasesLexicales(paradigmes)
    paradigmesCLIQUES=filtrerFidelite(cliques,casesLexicales)
    paradigmesSILVER=filtrerMax(paradigmesCLIQUES)
    paradigmesSILVER.to_csv(path_or_buf=nomSample.replace(".pkl","-Silver.csv"),encoding="utf8",sep=";")
    if nomGold:
        print (paradigmesSILVER[["known","right","wrong","missing"]].sum(),numSample,serie)

0 /Users/gilles/Box Sync/2015-Data/German/German-V-X-MSP.pkl
/Users/gilles/Box Sync/2015-Data/German/German-V-X-MSP.pkl
/Users/gilles/Box Sync/2015-Data/German/German-V-X-MSP-Regles.pkl
Attention l'analyse ne comprend pas toutes les cases
3282


# TESTS

In [ ]:
paradigmesSILVER[paradigmesSILVER["wrong"]>0]

In [ ]:
paradigmesBase[paradigmesBase["fs"].notnull() & paradigmesBase["fs"].str.contains(",")]

In [ ]:
lexique=lireLexique(nomSample.replace("OMP","MSP"))
lexique[lexique["lexeme"]=="lapon"]

In [ ]:
resultatsLecture[("fp","ms")].patrons

In [ ]:
pd.read_csv(path_or_buf=nomSample.replace(".pkl","-Silver.csv"),encoding="utf8",sep=";")

In [131]:
paradigmesSILVER[paradigmesSILVER["ajouts"]!="0"].groupby("NFIN").agg(' '.join)

,lexeme,PLxPRSxINDx2,PLxPRSxOPTx2,PLxPSTxCONDxPFVx1,PLxPSTxCONDxPFVx2,PLxPSTxINDxPFVx1,PLxPSTxINDxPFVx2,PRS,PST,SGxPRSxINDx1,SGxPRSxINDx2,SGxPRSxINDx3,SGxPRSxOPTx2,SGxPSTxCONDxPFVx1,SGxPSTxCONDxPFVx2,SGxPSTxINDxPFVx1,SGxPSTxINDxPFVx2,score,ajouts,lexicales
NFIN,,,,,,,,,,,,,,,,,,,,
,antwortetest atmeten ballastete ballten beabsi...,ballastet bearbeitet beiçtt beke...,ballet beakstet bearbeitet beiçtet...,ballten beabsiçtigten beakstten beaksteten ...,antvortetet atmetet beakstetet befrei...,antvorteten atmeten ballasteten beaksteten...,atmetet ballastetet beaksttet beakstetet b...,beabsiçtigend beiçtend bekehren...,antvortet beabsiçtigt begnadigt ...,ball6 beakst6 beakst6 bearbeit6 befolg6 ...,atmest ballastest beakstst bearbeitest ...,befolgt befyrçtet b...,bearbeitest bearbeitest beiçtest ...,ballastet6 beakstt6 beakstet6 bearbeitet6...,antvortetest balltest beakstetest befol...,beabsiçtigt6 beakstt6 befyrçtet6 b...,antvortetest beabsiçtigtest beakstetest ...,10.0 6.0 15.0 10.0 6.0 25.8125 5.8125 15.0 14....,2 1 3 2 1 5 1 3 3 3 1 2 4 3 1 6 2 5 2 1 3 1 1 ...,"PLxPSTxCONDxPFVx2, SGxPSTxINDxPFVx2 PLxPSTxIND..."
Etzen,ätze ätzten,Etzt,,Etzten Etzten,Etztet,Etzten Etzten,,Etzend,,Etz6,Etzst,,,Etzt6 Etzt6,Etztest,Etzt6,,36.0 15.0,7 3,"SGxPRSxINDx1 PLxPSTxCONDxPFVx2, PLxPSTxINDxPFVx1"
Eçten,ächtest,Eçtet,,,,Eçteten,,,,,Eçtest,,,,,,Eçtetest,15.0,3,"NFIN, SGxPRSxINDx2"
S2pfen,schöpfest schöpftest,S2pft,S2pfet S2pfet,S2pften S2pften,,S2pften S2pften,S2pftet,S2pfend,,S2pf6 S2pf6,,,S2pfest S2pfest,S2pft6 S2pft6,S2pftest S2pftest,S2pft6 S2pft6,S2pftest S2pftest,45.0 45.0,7 7,"SGxPRSxOPTx2, SGxPSTxINDxPFVx1 NFIN, SGxPSTxCO..."
SEmen,schämtet,SEmt,,,SEmtet,,,SEmend,,,,,,,,,,10.0,1,"PLxPRSxINDx2, PLxPSTxCONDxPFVx2, PRS"
SEnden,schändest,,SEndet,,,SEndten,,SEndend,,SEnd6,SEndst,,SEndest,SEndt6,,,,27.75,5,"NFIN, SGxPRSxOPTx2"
SErfen,schärft,SErft,SErfet,SErften,SErftet,SErften,,SErfend,,SErf6,SErfst,,SErfest,SErft6,,SErft6,SErftest,77.9722222222,10,"PLxPRSxINDx2, PLxPSTxCONDxPFVx1"
SEumen,schäumtet,SEumt,SEumet,SEumten,SEumtet,,,SEumend,,SEum6,,,,,,,,21.0,3,"PLxPRSxOPTx2, PLxPSTxCONDxPFVx2, SGxPRSxINDx1"
Saben,schabt,Sabt,,Sabten,,Sabten,,,,Sab6,,,,Sabt6,,,Sabtest,15.0,3,"PLxPRSxINDx2, PLxPSTxCONDxPFVx1"
